# Prepare the survey with the more restricted ingredient set

In [43]:
import os
import json
import pickle
import random

from generic_preprocessing import (make_recipes_extended_dict)

from calc_recipe_ingredient_info_distances import collectSomeRecipeRecommendations, get_all_comments, get_all_mutual_info, evalRecommendations, getRecommendationsBasedOnMutualInformationRole, get_graph_nodes, get_all_gt_recipes, get_recipes_per_ingredient, get_recipes_per_ingredient_pairs, get_all_frequencies,getNaiveBayesRecommendations

In [56]:
ORDERED_RECIPE_IDS_PATH = os.path.abspath("./outputs/sorted_recipe_ids_list.pkl")
TRAIN_COMMENTS_PATH = os.path.abspath("./inputs/train_comments_subs.pkl") # train recipes with substitutions
TEST_COMMENTS_PATH = os.path.abspath("./inputs/test_comments_subs.pkl") # test recipes with substitutions
VAL_COMMENTS_PATH = os.path.abspath("./inputs/val_comments_subs.pkl") # validation recipes with substitutions
GRAPH_NODES_PATH = os.path.abspath("./inputs/graph/nodes_191120.csv")

MUTUAL_INFO_DICT_PATH = os.path.abspath("./outputs/mutual_info_dict_with_self_info.pkl")
RECIPES_PER_INGREDIENT_SMALL_PATH = os.path.abspath(
    "./outputs/recipes_per_ingredient_small.pkl"
)
RECIPES_PER_INGREDIENT_PAIRS_SMALL_PATH = os.path.abspath(
    "./outputs/recipes_per_ingredient_pairs_small.pkl"
)
PROCESSED_RECIPES_PATH = os.path.abspath("./outputs/processed_recipes.pkl")
PATH_ONE_HOT_RECIPE_INGREDIENTS = os.path.abspath("./outputs/one_hot_recipe_ingredients.pkl")

EXTENDED_RECIPES_PATH_OLD = os.path.abspath("./inputs/extended_recipes_with_instructions_and_titles.json")

EXTENDED_RECIPES_PATH = os.path.abspath("./inputs/extended_recipes_with_quantities.json")


SURVEY_COMPLETE_SUB_TUPLE_AND_RECIPE_SET_PATH = os.path.abspath("./outputs/suvey_all_subs_and_recipes.json")

SURVEY_QUESTION_SET_A_PATH = os.path.abspath("./outputs/suvey_question_set_restrained_a.json")
SURVEY_QUESTION_SET_B_PATH = os.path.abspath("./outputs/suvey_question_set_restrained_b.json")
SURVEY_QUESTION_500_SET_PATH = os.path.abspath("./outputs/suvey_question_set_restrained_500.json")

ARCELIC_TO_R1M_RAW_PATH = os.path.abspath("./outputs/arcelic_raw_to_r1m.json")

In [3]:
ingredients = get_graph_nodes(GRAPH_NODES_PATH)

with open(SURVEY_COMPLETE_SUB_TUPLE_AND_RECIPE_SET_PATH, "r") as survey_data_path:
    survey_data = json.load(survey_data_path)

print(len(survey_data))

with open(ARCELIC_TO_R1M_RAW_PATH, "r") as arcelic_to_r1m_file:
    arcelic_to_r1m_raw = json.load(arcelic_to_r1m_file)

QUALTRICS_SURVEY_DATA_FILE_PATH = os.path.abspath("./inputs/survey_recipes_cgpt_suggestions.json")
with open(QUALTRICS_SURVEY_DATA_FILE_PATH, "r") as qualstrics_data_file:
    qualtrics_data = json.load(qualstrics_data_file)


1260


In [44]:
# recipes with comments, lists, names, and gt_truths
with open(EXTENDED_RECIPES_PATH, 'r') as recipe_extended_with_original_info:
    extended_recipes = json.load(recipe_extended_with_original_info)

recipes_extended_dict = make_recipes_extended_dict(extended_recipes)

## Check if all the parsed labels are present in FlavorGraph

In [4]:
# get set of distinct r1m ingredients from the mapping:
distinct_mapping_targets = sorted({x for v in arcelic_to_r1m_raw.values() for x in v})
print(distinct_mapping_targets[:10])

['10%_cream', '18%_table_cream', '2%_cheddar_cheese', '2%_evaporated_milk', '2%_milk', '2%_mozzarella_cheese', '35%_cream', '80%_lean_ground_beef', '85%_lean_ground_beef', '90%_lean_ground_beef']


In [5]:
found_labels = []
not_found_labels = []

for mapped_target in distinct_mapping_targets:
    if mapped_target in ingredients:
        found_labels.append(mapped_target)
    else:
        not_found_labels.append(mapped_target)

print(len(found_labels))
print(len(not_found_labels))

1804
0


## Enrich the survey candidate recipes

Check how many ingredients are in the mapping list, how many are not in that list but are in the ingredients list, and how many are in neither

In [6]:
for substitution_tuple, recipe in survey_data:
    sub_source, sub_target = substitution_tuple
    if "arcelic_familiar_sub_pair" not in list(recipe.keys()):
        if sub_source in distinct_mapping_targets and sub_target in distinct_mapping_targets:
            recipe["arcelic_familiar_sub_pair"] = True
        else:
            recipe["arcelic_familiar_sub_pair"] = False

    for recipe_ingredientrecipe in ["ingredients"]:
        ingredient_mapped = False

pass


In [7]:
# check the results:
# how many samples have sub pairs that are not mappable?
samples_with_mapped_sub_pairs = []
samples_with_not_mapped_sub_pairs = []
for sample in survey_data:
    substitution_tuple, recipe = sample
    if recipe["arcelic_familiar_sub_pair"]:
        samples_with_mapped_sub_pairs.append(sample)
    else:
        samples_with_not_mapped_sub_pairs.append(sample)

print(len(samples_with_mapped_sub_pairs))
print(len(samples_with_not_mapped_sub_pairs))

345
915


In [8]:
for substitution_tuple, recipe, sample_id in qualtrics_data:
    sub_source, sub_target = substitution_tuple
    if "arcelic_familiar_sub_pair" not in list(recipe.keys()):
        if sub_source in distinct_mapping_targets and sub_target in distinct_mapping_targets:
            recipe["arcelic_familiar_sub_pair"] = True
        else:
            recipe["arcelic_familiar_sub_pair"] = False

    for recipe_ingredientrecipe in ["ingredients"]:
        ingredient_mapped = False

# check the results:
# how many samples have sub pairs that are not mappable?
qsamples_with_mapped_sub_pairs = []
qsamples_with_not_mapped_sub_pairs = []
for sample in qualtrics_data:
    substitution_tuple, recipe, sample_id = sample
    if recipe["arcelic_familiar_sub_pair"]:
        qsamples_with_mapped_sub_pairs.append(sample)
    else:
        qsamples_with_not_mapped_sub_pairs.append(sample)

print(len(qsamples_with_mapped_sub_pairs))
print(len(qsamples_with_not_mapped_sub_pairs))


175
325


In [19]:
candidate_mapped_sub_pairs = [sp for sp, r in samples_with_mapped_sub_pairs]
already_selected_mapped_sub_pairs = [sp for sp, r, i in qsamples_with_mapped_sub_pairs]

unique_already_selected_mapped_sub_pairs = []
for sub_pair in already_selected_mapped_sub_pairs:
    if sub_pair not in unique_already_selected_mapped_sub_pairs:
        unique_already_selected_mapped_sub_pairs.append(sub_pair)

# print(candidate_mapped_sub_pairs)
# print(already_selected_mapped_sub_pairs)
potential_additional_sub_pairs = []
for sub_pair in candidate_mapped_sub_pairs:
    if sub_pair in potential_additional_sub_pairs:
        continue
    if sub_pair not in already_selected_mapped_sub_pairs:
        potential_additional_sub_pairs.append(sub_pair)

print(len(potential_additional_sub_pairs))

for sub_pair in potential_additional_sub_pairs:
    print(sub_pair)

print(len(unique_already_selected_mapped_sub_pairs))

# for sub_pair in unique_already_selected_mapped_sub_pairs:
#     print(sub_pair)

34
['paprika', 'chili_powder']
['ground_beef', 'roast_beef']
['walnut', 'slivered_almond']
['lemon', 'lime']
['green_pepper', 'red_bell_pepper']
['raisin', 'blueberry']
['diced_tomato', 'crushed_tomato']
['apple', 'applesauce']
['dry_white_wine', 'water']
['lean_ground_beef', 'ground_pork']
['lean_ground_beef', 'turkey_meat']
['white_wine', 'lemon_juice']
['semi_sweet_chocolate_chip', 'walnut']
['cooked_chicken', 'turkey']
['white_vinegar', 'white_wine_vinegar']
['chicken', 'turkey_breast']
['slivered_almond', 'walnut']
['red_wine', 'water']
['red_wine', 'white_wine']
['ground_turkey', 'lean_ground_beef']
['cherry_tomato', 'plum_tomato']
['unsweetened_applesauce', 'banana']
['pork_tenderloin', 'pork_loin_roast']
['fresh_tomato', 'diced_tomato']
['ground_chicken', 'turkey']
['flank_steak', 'sirloin_steak']
['chicken_piece', 'chicken_breast']
['extra_lean_ground_beef', 'ground_turkey']
['chicken_wing', 'chicken_leg']
['lean_ground_turkey', 'beef']
['deli_ham', 'bacon']
['walnut_piece', '

In [10]:
new_sub_tuples = [
    ['ground_beef', 'roast_beef'],
    ['dry_white_wine', 'water'],
    ['lean_ground_beef', 'ground_pork'],
    ['lean_ground_beef', 'turkey_meat'],
    ['white_wine', 'lemon_juice'],
    ['semi_sweet_chocolate_chip', 'walnut'],
    ['chicken', 'turkey_breast'],
    ['slivered_almond', 'walnut'],
    ['ground_turkey', 'lean_ground_beef'],
    ['unsweetened_applesauce', 'banana'],
    ['ground_chicken', 'turkey'],
    ['flank_steak', 'sirloin_steak'],
    ['extra_lean_ground_beef', 'ground_turkey'],
    ['lean_ground_turkey', 'beef'],
    ['deli_ham', 'bacon'],
]
len(new_sub_tuples)

15

In [11]:
new_survey_sub_pairs = [*unique_already_selected_mapped_sub_pairs, *new_sub_tuples]
print(len(new_survey_sub_pairs))
# for p in new_survey_sub_pairs:
#     print(p)

50


## Select samples where sub pair is suggested

- sub pair must be included in the sample as suggested for this recipe
- numbers with the least number of occurrences of ingredients that canont be matched are included first, then in order
- we aim for 7 recipes where the sub tuple was recommended, and fill up to 10 from other recipes
- if there are fewer than 7 recipes to choose from, use all of them and padd with more from the other recipes
- (Bonus: consider the argelic recipes first for padding up the rest)

In [32]:
ALL_SAMPLES_PER_ORIGINAL_RECOMMENDATION_PATH = os.path.abspath("./outputs/survey_candidate_samples.pkl")
with open(ALL_SAMPLES_PER_ORIGINAL_RECOMMENDATION_PATH, "rb") as samples_per_original_recommendation_file:
            all_samples_per_original_recommendation = pickle.load(samples_per_original_recommendation_file)


In [34]:
recipes_per_selected_sub_pair = {(sub_pair[0], sub_pair[1]): all_samples_per_original_recommendation[(sub_pair[0], sub_pair[1])] for sub_pair in new_survey_sub_pairs}
print(len(recipes_per_selected_sub_pair))

50


In [39]:
def sortRecipesByNrLeastUnmatchedIngredients(recipe_list):
    for recipe in recipe_list:
        nr_unmatched_ingredients = 0
        for ingredient in recipe["ingredients"]:
            ingredient_is_unmatched = True
            if isinstance(ingredient, list):
                for ingredient_variant in ingredient:
                    if ingredient_variant in distinct_mapping_targets:
                        ingredient_is_unmatched = False
            else:
                if ingredient in distinct_mapping_targets:
                    ingredient_is_unmatched = False
            if ingredient_is_unmatched:
                nr_unmatched_ingredients += 1
        recipe["nr_arcelic_unmatched_ingredients"] = nr_unmatched_ingredients

    recipe_list_sorted = sorted(recipe_list, key=lambda x: x["nr_arcelic_unmatched_ingredients"])
    return recipe_list_sorted

def sample_n_most_familiar_recipes(sorted_recipe_list, n):
    unmapped_arcelic_ingredients_count_target = sorted_recipe_list[n]["nr_arcelic_unmatched_ingredients"]
    viable_recipes = [
        recipe for recipe in sorted_recipe_list if recipe["nr_arcelic_unmatched_ingredients"] <= unmapped_arcelic_ingredients_count_target
        ]

    selected_recipes = random.sample(viable_recipes, n)
    return selected_recipes


In [42]:
n = 7

final_recipes_per_ingredient = {}
for sub_tuple, recipes in list(recipes_per_selected_sub_pair.items()):
    if len(recipes) <= n:
        final_recipes_per_ingredient[sub_tuple] = recipes
    else:
        final_recipes_per_ingredient[sub_tuple] = sample_n_most_familiar_recipes(sortRecipesByNrLeastUnmatchedIngredients(recipes), n)

print(len(final_recipes_per_ingredient))

50


### jetze musste ohno die alten rezepte durchwurschtln unde liste offülln


In [45]:
def getRecipesSubsAreIngredientsButNotGT(sub_pair, recipes_extended_dict):
    gt_source = sub_pair[0]
    gt_target = sub_pair[1]

    matched_recipes = []
    for recipe_id, recipe in list(recipes_extended_dict.items()):
        ingredients = recipe["ingredients"]
        has_gt_sub_recommendation = False
        subs_collection = recipe["subs_collection"]

        for sub in subs_collection:
            if isinstance(sub[0], list):
                sub_source = sub[0][0]
                sub_target = sub[0][1]
            else:
                sub_source = sub[0]
                sub_target = sub[1]
            if gt_source == sub_source and gt_target == sub_target:
                has_gt_sub_recommendation = True
                break

        if has_gt_sub_recommendation:
            continue

        for ingredient in ingredients:
            for ingredient_variant in ingredient:
                if ingredient_variant == gt_source:
                    matched_recipes.append(recipe)
                else:
                    continue
    return matched_recipes

In [49]:
N = 10

if N < n:
    raise RuntimeError("total number of requested recipes greater than number of gt recommended recipes")

for sub_pair, recipe_list in list(final_recipes_per_ingredient.items()):
    k = N - len(recipe_list) # fill with k recipes to have N recipes per sub tuple total

    alternative_recipe_options = getRecipesSubsAreIngredientsButNotGT(sub_pair, recipes_extended_dict)
    alternative_recipe_options_sorted = sortRecipesByNrLeastUnmatchedIngredients(alternative_recipe_options)
    additinoal_recipes = sample_n_most_familiar_recipes(alternative_recipe_options_sorted, k)
    final_recipes_per_ingredient[sub_pair] = [*recipe_list, *additinoal_recipes]

In [51]:
for sub_tuple, recipe_list in list(final_recipes_per_ingredient.items()):
    if len(recipe_list) != 10:
        print(sub_pair)

## Format the finaly samples into ye trusty olde json format

In [55]:
final_samples = []
id_counter = 0
for sub_tuple, recipe_list in list(final_recipes_per_ingredient.items()):
    for recipe in recipe_list:
        final_samples.append([
            [sub_tuple[0], sub_tuple[1]],
            recipe,
            id_counter
            ])
        id_counter += 1

random.shuffle(final_samples)

print(len(final_samples))

500


## finally print the stuff to the json

In [58]:
with open(SURVEY_QUESTION_500_SET_PATH, "w") as restrianed_samples_file:
    json.dump(final_samples, restrianed_samples_file, indent=2)